In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
import talib
import pandas as pd
import numpy as np
import cufflinks as cf
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import plotly.graph_objs as go
import warnings
import enum
warnings.filterwarnings('ignore')
init_notebook_mode(connected=True) 
import sqlite3

In [2]:
trading_list = {
    "IOC": {"price": 102.80, "leverage": 9, "reward": 1.5, "risk": 0.75},
    "TATAPOWER": {"price": 109.35, "leverage": 8, "reward": 1.5, "risk": 0.75},
    "ONGC": {"price": 115.10, "leverage": 9, "reward": 1.5, "risk": 0.75},
    "GAIL": {"price": 161.05, "leverage": 9, "reward": 1.5, "risk": 0.75},
    "ITC": {"price": 203.25, "leverage": 9, "reward": 2, "risk": 1},
    "HINDPETRO": {"price": 262.25, "leverage": 10, "reward": 1.5, "risk": 0.75},
    "DLF": {"price": 265.30, "leverage": 8, "reward": 1.5, "risk": 0.75},
    "AMBUJACEM": {"price": 309.05, "leverage": 9, "reward": 2, "risk": 1},
    "TATAMOTORS": {"price": 315.55, "leverage": 7, "reward": 2, "risk": 1},
    "HINDZINC": {"price": 317.10, "leverage": 10, "reward": 2, "risk": 1},
    "SBIN": {"price": 364.65, "leverage": 9, "reward": 2, "risk": 1},
    "WIPRO": {"price": 507.60, "leverage": 9, "reward": 2, "risk": 1},
    "DABUR": {"price": 529.20, "leverage": 9, "reward": 2, "risk": 1},
    "BHARTIARTL": {"price": 561.75, "leverage": 9, "reward": 2, "risk": 1},
    "ICICIBANK": {"price": 594.75, "leverage": 9, "reward": 2, "risk": 1},
    "TATACONSUM": {"price": 640.05, "leverage": 9, "reward": 2, "risk": 1}
}

ticker_list = list(trading_list.keys())


data_directory = "E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\"


# HINDPETRO = data[data["ticker"]=="HINDPETRO"]
# HINDZINC = data[data["ticker"]=="HINDZINC"].reset_index().drop(['index', 'ticker'], axis=1)
# SBIN = data[data["ticker"]=="SBIN"].reset_index().drop(['index', 'ticker'], axis=1)
# ICICIBANK = data[data["ticker"]=="ICICIBANK"].reset_index().drop(['index', 'ticker'], axis=1)
# WIPRO = data[data["ticker"]=="WIPRO"].reset_index().drop(['index', 'ticker'], axis=1)
# ITC = data[data["ticker"]=="ITC"].reset_index().drop(['index', 'ticker'], axis=1)
# IOC = data[data["ticker"]=="IOC"].reset_index().drop(['index', 'ticker'], axis=1)
# ONGC = data[data["ticker"]=="ONGC"].reset_index().drop(['index', 'ticker'], axis=1)
# DABUR = data[data["ticker"]=="DABUR"].reset_index().drop(['index', 'ticker'], axis=1)
# GAIL = data[data["ticker"]=="GAIL"].reset_index().drop(['index', 'ticker'], axis=1)
# TATACONSUM = data[data["ticker"]=="TATACONSUM"].reset_index().drop(['index', 'ticker'], axis=1)
# AMBUJACEM = data[data["ticker"]=="AMBUJACEM"].reset_index().drop(['index', 'ticker'], axis=1)
# DLF = data[data["ticker"]=="DLF"].reset_index().drop(['index', 'ticker'], axis=1)
# TATAPOWER = data[data["ticker"]=="TATAPOWER"].reset_index().drop(['index', 'ticker'], axis=1)
# TATAMOTORS = data[data["ticker"]=="TATAMOTORS"].reset_index().drop(['index', 'ticker'], axis=1)

In [3]:
def graph(data,ticker):
    graph_data = pd.DataFrame(columns=['x','Marker','Symbol','Color'])
    hammer = talib.CDLHAMMER(data["open"], data["high"], data["low"], data["close"])
    data['Hammer'] = hammer
    for index, row in data.iterrows():
        ticker_data = {}        
        val = row["Hammer"]
        if val == 100:                             
            ticker_data["x"] = index
            ticker_data["Marker"] = row["close"]
            ticker_data["Symbol"] ="triangle-up"
            ticker_data["Color"] ="green"
            graph_data = graph_data.append(ticker_data, ignore_index=True)  
    Candle = go.Candlestick(x=data.index,
                       open=data.open,
                       high=data.high,
                       low=data.low,
                       close=data.close
                       )  
    Trace = go.Scatter(x=graph_data.x,
                       y=graph_data.Marker,
                       mode='markers',
                       name ='markers',
                       marker=go.Marker(size=20,
                                        symbol=graph_data["Symbol"],
                                        color=graph_data["Color"])
                   )
    iplot([Candle, Trace])

In [4]:
def spread(data,ticker):
    data.iplot(kind="spread", keys=["open", "close",],
               title="Open & Close Price Ratio Chart")

In [5]:
def candle_graph(data,ticker):
    qf=cf.QuantFig(data,rangeslider=True,title='EMA ',legend='top',name=ticker)
    qf.add_volume()
    qf.iplot()

In [6]:
class StatisticsManager:

    def __init__(self):
        directory = data_directory + "db\\morning_star\\"
        if not os.path.exists(directory):
            os.makedirs(directory)
        self.name =directory+ "statistics.db"        
        self.createStatistics()

    def connectDB(self):
        self.db = sqlite3.connect(self.name)

    def createStatistics(self):
        self.connectDB()
        cursor = self.db.cursor()
        query = "CREATE TABLE IF NOT EXISTS Statistics (id INTEGER PRIMARY KEY, open_time DATETIME NOT NULL, symbol TEXT NOT NULL, algorithm TEXT NOT NULL, number_of_shares INTEGER, type TEXT NOT NULL, current REAL(15,5), reward REAL(15,5), risk REAL(15,5), close REAL(15,5), status TEXT NOT NULL, close_time DATETIME)"
        cursor.execute(query)
        try:
            self.db.commit()
        except Exception as e:
            print(e)
            self.db.rollback()
        self.close()

    def insertStatistics(self,data):
        self.connectDB()
        cursor = self.db.cursor()
        vals = [data['open_time'],
                data['symbol'],
                data['algorithm'],
                data['count'],
                data['type'],
                data['current'],
                data['reward'],
                data['risk'],
                data['close'],
                data['status'],
                data['close_time']
                ]
        query = "INSERT INTO Statistics(open_time, symbol, algorithm, number_of_shares, type, current, reward, risk, close, status, close_time) VALUES (?,?,?,?,?,?,?,?,?,?,?)"
        cursor.execute(query, vals)
        try:
            self.db.commit()
        except Exception as e:
            print(e)
            self.db.rollback()
        self.close()

    def updateStatistics(self, data):
        self.connectDB()
        cursor = self.db.cursor()
        vals = [data['status'],
                data['close'],
                data['close_time'],
                data['symbol'],
                data['algorithm'],
                data['type']
                ]
        query = "UPDATE Statistics  SET status = ?, close =? , close_time = ? WHERE symbol = ? AND algorithm = ? AND type=? AND status = 'open' "
        cursor.execute(query, vals)
        try:
            self.db.commit()
        except Exception as e:
            print(e)
            self.db.rollback()
        self.close()


    def close(self):
        self.db.close()

    def deleteDB(self,db):
        if os.path.exists(db):
            os.remove(db)
        else:
            print("The file does not exist")

In [7]:
class Candle(enum.Enum):
    red = 1
    green = 2
    doji = 3

In [8]:
class Trend(enum.Enum):
    strong_uptrend = 1
    uptrend = 2
    strong_downtrend = 3
    downtrend = 4
    uptrend_reversal = 5
    downtrend_reversal = 6
    strong_uptrend_reversal = 7
    strong_downtrend_reversal = 8
    sidetrend = 9
       

In [9]:
class MoneyManager:
    def __init__(self):
        self.parallelTrade = 3
        self.wallet = 5000
        self.leverage = 9
        self.tradingList = {}


    def getNumberofShare(self,price):
        return self.calculateShare(price,self.leverage)


    def getNumberFromTicker(self,ticker):
        data = self.tradingList.get(ticker, None)
        if data["price"] == None:
            return -1
        return self.calculateShare(data["price"],data["leverage"])


    def calculateShare(self,price,leverage):
        return round((self.wallet * leverage) / (price * self.parallelTrade)) - 1

In [10]:
class CandleHelper:

    def __init__(self, count=4):
        self.count = count
        self.index = 0

    def is_doji_candle(self, candle):
        return candle == Candle.doji

    def is_green_candle(self, candle):
        return candle == Candle.green

    def is_red_candle(self, candle):
        return candle == Candle.red

    def get_last_candle(self, data):
        if data['close'].values > data['open'].values:
            return Candle.green
        if data['close'].values < data['open'].values:
            return Candle.red
        return Candle.doji

    def get_candle_details(self, data):
        candle_data = data.tail(4)
        current = self.get_last_candle(candle_data.tail(1))
        previous = self.get_last_candle(candle_data.tail(2).head(1))
        last_two = self.get_last_candle(candle_data.tail(3).head(1))
        last_three = self.get_last_candle(candle_data.head(1))
        return current, previous, last_two, last_three

    def trend_strength(self, data, trend):
        candle_data = data.tail(4)
        current = candle_data.tail(1)
        previous = candle_data.tail(2).head(1)
        last_two = candle_data.tail(3).head(1)
        last_three = candle_data.head(1)
        reversal_value = 0
        if trend == Trend.strong_uptrend:
            trend_value = abs(round((last_three['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.uptrend:
            trend_value = abs(round((last_two['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.strong_downtrend:
            trend_value = abs(round((last_three['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.downtrend:
            trend_value = abs(round((last_two['close'].values - current['open'].values)[0], 2))
        elif trend == Trend.uptrend_reversal:
            trend_value = abs(round((last_two['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.downtrend_reversal:
            trend_value = abs(round((last_two['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.strong_uptrend_reversal:
            trend_value = abs(round((last_three['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        elif trend == Trend.strong_downtrend_reversal:
            trend_value = abs(round((last_three['close'].values - previous['open'].values)[0], 2))
            reversal_value = abs(round((current['close'].values - previous['open'].values)[0], 2))
        else:
            trend_value = 0
            reversal_value = 0
        return trend_value, reversal_value

    def get_trends(self, data):
        current, previous, last_two, last_three = self.get_candle_details(data)
        trend = Trend.sidetrend
        if (self.is_green_candle(current) or self.is_doji_candle(current)) and (
                self.is_green_candle(previous) or self.is_doji_candle(previous)):
            if self.is_green_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_green_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_uptrend
                else:
                    trend = Trend.uptrend
        elif (self.is_red_candle(current) or self.is_doji_candle(current)) and (
                self.is_red_candle(previous) or self.is_doji_candle(previous)):
            if self.is_red_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_red_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_downtrend
                else:
                    trend = Trend.downtrend
        elif self.is_red_candle(current) and (self.is_green_candle(previous) or self.is_doji_candle(previous)):
            if self.is_green_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_green_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_uptrend_reversal
                else:
                    trend = Trend.uptrend_reversal
        elif self.is_green_candle(current) and (self.is_red_candle(previous) or self.is_doji_candle(previous)):
            if self.is_red_candle(last_two) or self.is_doji_candle(last_two):
                if self.is_red_candle(last_three) or self.is_doji_candle(last_three):
                    trend = Trend.strong_downtrend_reversal
                else:
                    trend = Trend.downtrend_reversal
        else:
            trend = Trend.sidetrend
        trend_value, reversal_value = self.trend_strength(data, trend)
        return trend, trend_value, reversal_value

    def get_next_candle(self,data):
        candle_data = data.tail(1)
        self.index +=1
        return self.get_last_candle(candle_data)

In [11]:
class BasePricePattern:

    def __init__(self, share, count, reward, risk, algorithm):
        self.date = ""
        self.data = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume', 'traded'])
        self.long_trade_list = []
        self.short_trade_list = []
        self.current_price = 0
        self.share = share
        self.count = count
        self.algorithm = algorithm
        self.reward = reward
        self.risk = risk
        self.statistic_manager = StatisticsManager()
        self.candle_helper = CandleHelper()
        self.next_long_pattern = False
        self.next_short_pattern = False
        self.long_open = False
        self.short_open = False

    def createTradeHistoryLong(self):
        data = {'open_time': self.date,
                'symbol': self.share,
                'algorithm': self.algorithm,
                'count': self.count,
                'type': "long",
                'current': self.current_price,
                'reward': (self.current_price + self.reward),
                'risk': (self.current_price - self.risk),
                'close': 0,
                'status': "open",
                'close_time': ""
                }
        self.long_open = True
        self.long_trade_list.append(data)
        self.statistic_manager.insertStatistics(data)

    def createTradeHistoryShort(self):
        data = {'open_time': self.date,
                'symbol': self.share,
                'algorithm': self.algorithm,
                'count': self.count,
                'type': "short",
                'current': self.current_price,
                'reward': (self.current_price - self.reward),
                'risk': (self.current_price + self.risk),
                'close': 0,
                'status': "open",
                'close_time': ""
                }
        self.short_open = True
        self.short_trade_list.append(data)
        self.statistic_manager.insertStatistics(data)

    def closeTrade(self, current_price):
        if len(self.long_trade_list) > 0:
            row_l = self.long_trade_list[len(self.long_trade_list) - 1]
            if row_l['status'] == "open":
                if row_l['reward'] <= current_price:
                    row_l['close'] = current_price
                    row_l['status'] = "profit"
                    row_l['close_time'] = self.date
                    self.update_trade_statistic(row_l)
                if row_l['risk'] >= current_price:
                    row_l['close'] = current_price
                    row_l['status'] = "loss"
                    row_l['close_time'] = self.date
                    self.update_trade_statistic(row_l)

        if len(self.short_trade_list) > 0:
            row_s = self.short_trade_list[len(self.short_trade_list) - 1]
            if row_s['status'] == "open":
                if row_s['reward'] >= current_price:
                    row_s['close'] = current_price
                    row_s['status'] = "profit"
                    row_s['close_time'] = self.date
                    self.update_trade_statistic(row_s)
                if row_s['risk'] <= current_price:
                    row_s['close'] = current_price
                    row_s['status'] = "loss"
                    row_s['close_time'] = self.date
                    self.update_trade_statistic(row_s)

    def closeAllTrades(self):
        val = self.data["close"].values[0]
        if len(self.long_trade_list) > 0:
            row_l = self.long_trade_list[len(self.long_trade_list) - 1]
            if row_l['status'] == "open":
                row_l['close'] = val
                row_l['status'] = "closed"
                row_l['close_time'] = self.date
                self.update_trade_statistic(row_l)

        if len(self.short_trade_list) > 0:
            row_s = self.short_trade_list[len(self.short_trade_list) - 1]
            if row_s['status'] == "open":
                row_s['close'] = val
                row_s['status'] = "closed"
                row_s['close_time'] = self.date
                self.update_trade_statistic(row_s)

    def update_trade_statistic(self, data):
        data = {
            'status': data['status'],
            'close': data['close'],
            'close_time': data['close_time'],
            'symbol': data['symbol'],
            'algorithm': data['algorithm'],
            'type': data['type']
        }
        self.statistic_manager.updateStatistics(data)

    def trade(self, data, date):
        self.date = date
        self.data = self.data.append(data, ignore_index=True)
        if self.long_open:
            self.close_long_trade()
        elif self.short_open:
            self.close_short_trade()
        else:
            if self.next_long_pattern:
                self.long_pattern()
            else:
                self.long_trade()
            if self.next_short_pattern:
                self.short_pattern()
            else:
                self.short_trade()

    def long_pattern(self):
        if self.candle_helper.index > 3:
            self.candle_helper.index = 0
            self.next_long_pattern = False
            self.long_trade()
        else:
            pattern = self.candle_helper.get_next_candle(self.data.tail(1))
            if pattern == Candle.green:
                self.candle_helper.index = 0
                self.next_long_pattern = False
                self.long_trade(long_pattern=True)

    def short_pattern(self):
        if self.candle_helper.index > 3:
            self.candle_helper.index = 0
            self.next_short_pattern = False
            self.short_trade()
        else:
            pattern = self.candle_helper.get_next_candle(self.data.tail(1))
            if pattern == Candle.red:
                self.candle_helper.index = 0
                self.next_short_pattern = False
                self.short_trade(short_pattern=True)

    def long_trade(self, long_pattern=False):
        pass

    def short_trade(self, short_pattern=False):
        pass


    def close_long_trade(self):
        pass

    def close_short_trade(self):
        pass

In [12]:
class BullishMorningStar(BasePricePattern):

    def __init__(self, share, count, reward, risk,spread_limit =  0.15):
        super().__init__(
            share,
            count,
            reward,
            risk,
            self.__class__.__name__
        )
        self.spread_limit = spread_limit

    def long_trade(self, long_pattern=False):
        if long_pattern:
            self.current_price = self.data["close"].values[0]
            self.createTradeHistoryLong()
        else:
            self.morning_star()
            data = self.data
            morning_star = data[(data['MorningStar'] == 100) & (data['traded'] == False)].tail(1)

            if len(morning_star) > 0:
                self.data.loc[self.data['MorningStar'] == 100, 'traded'] = True
                self.current_price = morning_star['close'].values[0]
                if not morning_star['traded'].values[0]:
                    trend, trend_value, reversal_value = self.candle_helper.get_trends(self.data.tail(4))
                    if trend == Trend.downtrend_reversal or trend == Trend.strong_downtrend_reversal:
                        self.createTradeHistoryLong()
                    else:
                        self.candle_helper.index = 0
                        self.next_long_pattern = True

    def morning_star(self):
        engulfing = talib.CDLMORNINGSTAR(self.data['open'].values, self.data["high"], self.data["low"], self.data["close"])
        self.data['MorningStar'] = engulfing

    def close_long_trade(self):
        candle_data = self.data.tail(1)
        closing_price = candle_data['close'].values[0]
        spread = round(abs(candle_data['open'].values[0]  - closing_price),2)
        pattern = self.candle_helper.get_next_candle(candle_data)
        if (pattern == Candle.red and spread >= self.spread_limit ) or closing_price <= self.current_price - self.risk :
            data_long = self.long_trade_list[len(self.long_trade_list) - 1]
            if data_long['status'] == "open":
                data_long['close'] = closing_price
                data_long['close_time'] = self.date
                if closing_price > self.current_price:
                    data_long['status'] = "profit"
                else:
                    data_long['status'] = "loss"
                self.update_trade_statistic(data_long)

In [13]:
def trade_last_60(ticker, startday, data):
    manager = MoneyManager()
    manager.wallet = 16000
    manager.leverage = 9
    manager.parallelTrade = 3
    manager.tradingList = trading_list
    trading = BullishMorningStar(ticker, manager.getNumberFromTicker(ticker),1,0.5)
    trade_data = pd.DataFrame(columns = ['open', 'high', 'low','close','volume','traded'])
    for index, row in data.iterrows():
        row_data = {}
        row_data["open"] = row["open"]
        row_data["close"] = row["close"]
        row_data["low"] = row["low"]
        row_data["high"]= row["high"]
        row_data["volume"] = row["volume"]
        row_data["traded"] = False
        #trade_data = trade_data.append(row_data, ignore_index=True)
        if len(row_data) > 0:
            trading.trade(row_data,startday)
        else:
            print("hammer trade_data is empty")
    return trading.data

In [14]:
def process_last_60(date,offset):   
    print("day : ",offset)
    startday = date.strftime('%Y-%m-%d-%H:%M:%S')
    directory = data_directory+"last_60\\17_05_2021\\days\\"
    file_name = directory + str(offset) + ".csv"
    data = pd.read_csv(file_name)
    process_data = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume', 'traded'])
    for ticker in ticker_list:        
        daydata = trade_last_60(ticker, startday, data[data["ticker"]==ticker])
        daydata['ticker'] = ticker
        daydata['date'] = startday
        process_data = process_data.append(daydata, ignore_index=True)
    return process_data

In [15]:
def start_last_60_days():
    days_to_subtract = 38
    startday = datetime.today() - timedelta(days=days_to_subtract)
    start_data = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume', 'traded'])
    for offset in range(1,39):
        offset_data=process_last_60(startday,offset)
        start_data = start_data.append(offset_data, ignore_index=True)        
        startday = startday + timedelta(days=1)
    return start_data

In [16]:
def save(_data):
    directory = data_directory+"test_data\\morning_star\\"
    if not os.path.exists(directory):
        os.makedirs(directory)
    for ticker in ticker_list: 
        file_name = directory+ticker+".csv"
        _data[_data["ticker"]==ticker].to_csv(file_name)

In [17]:
data = pd.read_csv("E:\\Workspace\\Python\\PythonPOC\\notebook\\data\\last_60\\17_05_2021\\days\\1.csv")
startday = datetime.today().strftime('%Y-%m-%d-%H:%M:%S')
HINDPETRO = data[data["ticker"]=="HINDPETRO"]
len(HINDPETRO)

375

In [18]:
_data =start_last_60_days()
save(_data)


day :  1
day :  2
day :  3
day :  4
day :  5
day :  6
day :  7
day :  8
day :  9
day :  10
day :  11
day :  12
day :  13
day :  14
day :  15
day :  16
day :  17
day :  18
day :  19
day :  20
day :  21
day :  22
day :  23
day :  24
day :  25
day :  26
day :  27
day :  28
day :  29
day :  30
day :  31
day :  32
day :  33
day :  34
day :  35
day :  36
day :  37
day :  38
